# Wander Wise - AI Trip Planner

**User input：**

In [ ]:
location = "Manhattan, New York, NY"
budget = "1000 USD"
days = "5 days"
interests = "Restaurant"

**Install necessary libraries:**

In [ ]:
!pip install polyline

In [ ]:
pip install requests

In [ ]:
pip install openai==0.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install requests folium
import requests
import folium
from folium.plugins import MarkerCluster

**Return places from Yelp based on inputs**

In [ ]:
def get_top_places_from_yelp(location, interests, total_limit=10):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": "YOUR_OWN_API"
    }

    review_threshold = 50

    places = set()

    def fetch_places(term, limit):
        params = {"term": term, "location": location, "limit": limit}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'businesses' in data:
            for b in data['businesses']:
                if b.get('review_count', 0) >= review_threshold:
                    place = f"{b['name']} ({', '.join(b['location']['display_address'])})"
                    places.add(place)
                    if len(places) >= total_limit:
                        return

    for interest in interests:
        fetch_places(interest, 2)

    general_limit = total_limit - len(places)
    if general_limit > 0:
        fetch_places("tourist attractions", general_limit)

    return list(places)[:total_limit]

Backup Option

In [ ]:
import requests

def get_top_places_from_yelp(location, interests, total_limit=10):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": "YOUR_OWN_API"
    }

    review_threshold = 50  # Review threshold

    places = set()

    def fetch_places(term, limit):
        params = {"term": term, "location": location, "limit": limit}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'businesses' in data:
            for b in data['businesses']:
                if b.get('review_count', 0) >= review_threshold:
                    # More robust location parsing
                    location_parts = [part.strip() for part in location.split(",")]
                    target_city = location_parts[0].lower()
                    target_state = location_parts[1].lower() if len(location_parts) > 1 else ''

                    business_city = b['location'].get('city', '').strip().lower()
                    business_state = b['location'].get('state', '').strip().lower()

                    # Check if the business location matches the target location
                    if business_city == target_city and (not target_state or business_state == target_state):
                        place = f"{b['name']} ({', '.join(b['location']['display_address'])})"
                        places.add(place)
                        if len(places) >= total_limit:
                            return

    for interest in interests:
        fetch_places(interest, 1)

    general_limit = total_limit - len(places)
    if general_limit > 0:
        fetch_places("tourist attractions", general_limit)

    return list(places)[:total_limit]


**Save input from Yelp as locations**

In [ ]:
locations = get_top_places_from_yelp(location, interests)
locations

['Anytime Kitchen  (23 W 32nd St, Fl 3, New York, NY 10001)',
 'La Grande Boucherie (145 W 53rd St, New York, NY 10019)',
 'R Lounge at Two Times Square (Two Times Square, 714 Seventh Avenue at W. 48th Street, New York, NY 10036)',
 'S Wan Cafe 洋紫荆 (85 Eldridge St Lower E, New York, NY 10002)',
 'U and Me Nails (1465 2nd Ave, New York, NY 10075)',
 'Trattoria Trecolori (254 West 47th St, New York, NY 10036)',
 'Spyscape (928 8th Ave, New York, NY 10019)',
 "e's BAR (511 Amsterdam Ave, New York, NY 10024)",
 "Jacob's Pickles (509 Amsterdam Ave, New York, NY 10024)",
 'S&P (174 5th Ave, New York, NY 10010)']

**Use Google Map API to convert locations into coordinates**

In [ ]:
import os
import requests
import folium
from folium.plugins import MarkerCluster

API_KEY = 'YOUR_OWN_API'

# Get coordinates for a specific location using Google Maps Geocoding API
def get_coordinates(location):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

for loc in locations:
    coord = get_coordinates(loc)
    if coord:
        print(f"{loc}: {coord}")

Anytime Kitchen  (23 W 32nd St, Fl 3, New York, NY 10001): (40.7477749, -73.9867579)
La Grande Boucherie (145 W 53rd St, New York, NY 10019): (40.7626274, -73.98084109999999)
R Lounge at Two Times Square (Two Times Square, 714 Seventh Avenue at W. 48th Street, New York, NY 10036): (40.7596193, -73.9846788)
S Wan Cafe 洋紫荆 (85 Eldridge St Lower E, New York, NY 10002): (40.7173167, -73.9926444)
U and Me Nails (1465 2nd Ave, New York, NY 10075): (40.771602, -73.95658279999999)
Trattoria Trecolori (254 West 47th St, New York, NY 10036): (40.7599972, -73.9867421)
Spyscape (928 8th Ave, New York, NY 10019): (40.7652697, -73.9837269)
e's BAR (511 Amsterdam Ave, New York, NY 10024): (40.7867058, -73.9754856)
Jacob's Pickles (509 Amsterdam Ave, New York, NY 10024): (40.786648, -73.9755259)
S&P (174 5th Ave, New York, NY 10010): (40.7411077, -73.9901838)


**Sort the locations using a greedy algorithm and draw**

In [ ]:
import requests
import folium
from folium.plugins import MarkerCluster
import polyline

def get_coordinates(location):
    # Fetches geographic coordinates (latitude, longitude) for a given location
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY  # Replace with your Google Maps API key
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

def tsp_greedy(matrix):
    # Implements a greedy algorithm for solving the Traveling Salesman Problem (TSP)
    num_locations = len(matrix)
    unvisited = set(range(num_locations))
    path = [0]
    unvisited.remove(0)

    while unvisited:
        last = path[-1]
        next_loc = min(unvisited, key=lambda x: matrix[last][x])
        path.append(next_loc)
        unvisited.remove(next_loc)
    # path.append(path[0])
    return path

def get_chunked_distance_matrix(locations):
    # Fetches a distance matrix in chunks due to API limitations
    max_elements = 10  # Google's limitation is typically 10 for the free tier
    num_locations = len(locations)
    matrix = [[0 for _ in range(num_locations)] for _ in range(num_locations)]

    for i in range(0, num_locations, max_elements):
        for j in range(0, num_locations, max_elements):
            chunked_origins = locations[i:i+max_elements]
            chunked_destinations = locations[j:j+max_elements]
            chunk_matrix = get_distance_matrix(chunked_origins, chunked_destinations)

            if chunk_matrix is None:
                print("Error fetching a chunk of the distance matrix.")
                return None

            for x, row in enumerate(chunk_matrix):
                for y, distance in enumerate(row):
                    matrix[i+x][j+y] = distance
    return matrix

def get_distance_matrix(origins, destinations=None):
    # Fetches the distance matrix for given origins and destinations
    if destinations is None:
        destinations = origins

    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": "|".join(origins),
        "destinations": "|".join(destinations),
        "key": API_KEY  # Replace with your Google Maps API key
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        matrix = [[row['distance']['value'] for row in entry['elements']] for entry in data['rows']]
        return matrix
    else:
        print(f"Error fetching distance matrix. Error: {data['status']}")
        return None

def plot_with_actual_roads(locations, tuple_coordinates):
    # Check if locations and coordinates are not empty
    if not locations or not tuple_coordinates:
        print("No locations or coordinates provided.")
        return None

    # Initialize the map with the first location in the optimized list
    base_coord = tuple_coordinates[0]
    m = folium.Map(location=base_coord, zoom_start=13)
    marker_cluster = MarkerCluster().add_to(m)

    # Adding markers for each location in the optimized list
    for i, (location, coord) in enumerate(zip(locations, tuple_coordinates)):
        html = f'''
        <div style="background-color: blue; color: white; font-size: 12pt;
                    width: 24px; height: 24px; text-align: center;
                    border-radius: 50%; line-height: 24px;">
            {i+1}
        </div>
        '''
        icon = folium.DivIcon(html=html)

        folium.Marker(
            location=coord,
            popup=location,
            icon=icon
        ).add_to(marker_cluster)

    # Drawing road routes between consecutive locations
    road_path = []
    for i in range(len(tuple_coordinates) - 1):
        start_coord = tuple_coordinates[i]
        end_coord = tuple_coordinates[i + 1]
        segment_path = get_road_path(start_coord, end_coord)
        if segment_path:
            road_path.extend(segment_path)

    # Add a PolyLine to connect all locations without the first point
    folium.PolyLine(road_path, color="blue", weight=2.5, opacity=1).add_to(m)

    return m

def get_road_path(start_coord, end_coord):
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_coord[0]},{start_coord[1]}",
        "destination": f"{end_coord[0]},{end_coord[1]}",
        "key": API_KEY
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        # Extract polyline encoded string
        polyline_encoded = data['routes'][0]['overview_polyline']['points']
        # Decode polyline to a list of latitude and longitude pairs
        road_path = polyline.decode(polyline_encoded)
        return road_path
    else:
        print(f"Error fetching road path. Error: {data['status']}")
        return None

# Main script execution
full_locations = locations  # Use only the locations you want to optimize
coordinates = [get_coordinates(loc) for loc in full_locations]

distance_matrix = get_chunked_distance_matrix(full_locations)
if distance_matrix is None:
    print("Error getting distance matrix. Execution stopped.")
else:
    try:
        optimized_order = tsp_greedy(distance_matrix)
        optimized_coords = [coordinates[i] for i in optimized_order]
        optimized_locations = [full_locations[i] for i in optimized_order]

        map_object = plot_with_actual_roads(optimized_locations, optimized_coords)
        display(map_object)
    except KeyError as e:
        print(f"An error occurred: {e}")

**Use GPT to generate plans for the arranged locations**

In [ ]:
optimized_locations

['Anytime Kitchen  (23 W 32nd St, Fl 3, New York, NY 10001)',
 'S&P (174 5th Ave, New York, NY 10010)',
 'Trattoria Trecolori (254 West 47th St, New York, NY 10036)',
 'R Lounge at Two Times Square (Two Times Square, 714 Seventh Avenue at W. 48th Street, New York, NY 10036)',
 'Spyscape (928 8th Ave, New York, NY 10019)',
 'La Grande Boucherie (145 W 53rd St, New York, NY 10019)',
 "Jacob's Pickles (509 Amsterdam Ave, New York, NY 10024)",
 "e's BAR (511 Amsterdam Ave, New York, NY 10024)",
 'U and Me Nails (1465 2nd Ave, New York, NY 10075)',
 'S Wan Cafe 洋紫荆 (85 Eldridge St Lower E, New York, NY 10002)']

In [ ]:
import openai

def generate_plan(locations, budget, days, interest):
    """
    Generate a detailed travel plan based on a list of locations, budget, days, and interest using OpenAI's GPT-4 API.
    :param locations: List of locations for the travel plan
    :param budget: Budget for the trip
    :param days: Number of days of the trip
    :param interest: Specific interests for the trip
    :return: Generated travel plan
    """
    openai.api_key = "YOUR_OWN_API"

    # Build messages for the chat completion request
    messages = [
        {"role": "system", "content": "You are a helpful assistant designed to generate a travel plan in plain text format."},
        {"role": "user", "content": f"Create a detailed, budget-conscious travel plan based on the following ordered locations: {', '.join(optimized_locations)}. Budget: {budget}, Days: {days}, Interest: {interest}. The plan should include daily activities: Breakfast, Morning Attraction, Lunch, Afternoon Attraction, Dinner, Hotel and Accommodation. Calculate total daily expenses and ensure they do not exceed the allocated daily budget."}
    ]

    # Invoke the OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",  # Using GPT-4 preview model
        messages=messages
    )

    # Extract the generated text
    generated_text = response['choices'][0]['message']['content']

    return generated_text

plan = generate_plan(optimized_locations, budget, days, interests)
print(plan)


To create a travel plan that includes daily activities such as Breakfast, Morning Attraction, Lunch, Afternoon Attraction, Dinner, and Hotel and Accommodation, I will assume that your interest in 'Restaurant' implies a focus on culinary experiences. I will make conservative estimates for the costs based on moderate pricing and look for accommodation within a reasonable range to the points of interest.

Here's a budget-conscious, five-day travel plan in New York City:

**Day 1:**
- **Breakfast:** Anytime Kitchen (estimated cost: $15)
- **Morning Attraction:** Explore the Empire State Building (admission is around $44; however, if we skip to keep expenses low).
- **Lunch:** S&P (estimated cost: $20)
- **Afternoon Attraction:** Take a stroll in the Madison Square Park (free).
- **Dinner:** Trattoria Trecolori (estimated cost: $30)
- **Hotel:** Pod Times Square Hotel (budget option near attractions, approximately $150 per night including taxes).
- **Total Expenses for Day 1:** $259

**Day 